## ML Model for Dynamic Characters

### Imports

In [1]:
import keras
import numpy as np

In [2]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### Read Data

In [3]:
# get data 
# img = imageio.imread('../resources/textures/sphereTex.png')
# img = img[:, :, 0] # convert to grayscale
# rows = len(img) # rows
# cols = len(img[0]) # cols

# read data
# data = []
# with open("out.txt", "r") as text_file:
#     line = text_file.read()
#     data = line.split('\n')[:-1]
#     data = [ list(map(float, x.split(','))) for x in data ]
# data = np.array(data)

# # split data
# np.random.shuffle(data)
# partition = int(0.8 * len(data))
# X_train = data[:partition, :-1]
# y_train = data[:partition, -1] / 255.0
# X_test = data[partition:, :-1] 
# y_test = data[partition:, -1] / 255.0

In [5]:
# read data
name = "warrior"
train_path = "../resources/occlusion/_" + name + "_train_data.txt"
test_path = "../resources/occlusion/_" + name + "_test_data.txt"

data = []
bones = 0
vertices = 0
with open(train_path, "r") as text_file:
    raw = text_file.read()
    data = raw.split('\n')[:-1] # ignore empty list
    data = [ list(map(float, x.split(' '))) for x in data ]
    bones, vertices = data[0]
train_data = np.array(data[1:])

with open(test_path, "r") as text_file:
    raw = text_file.read()
    data = raw.split('\n')[:-1]
    data = [ list(map(float, x.split(' '))) for x in data ]
    bones, vertices = data[0]
test_data = np.array(data[1:])

# split data
x_len = int(3 * bones) # 3 for each euler angle

np.random.shuffle(train_data)
assert(len(train_data[0]) == x_len + vertices)
X_train = train_data[:, :x_len]
X_train = X_train / np.pi  #/ (2 * np.pi) # standardize
y_train = train_data[:, x_len:]

assert(len(test_data[0]) == x_len + vertices)
np.random.shuffle(test_data)
X_test = test_data[:, :x_len]
X_test = X_test / np.pi #/ (2 * np.pi) # standardize
y_test = test_data[:, x_len:]

### Build Model

<b>Inputs:</b> u, v, thetas
<br/><b>Output</b>: occlusion factor

In [19]:
# build model
model = keras.Sequential([
    keras.layers.Dense(256, input_shape=(x_len,), activation='relu'), # hidden
    keras.layers.Dense(256, activation='relu'), # hidden
    keras.layers.Dense(256, activation='relu'), # hidden
    keras.layers.Dense(vertices) # output
])
# model = keras.Sequential([
#     keras.layers.Conv1D(512, 15, 3, input_shape=(x_len,1,)), # hidden
#     keras.layers.Conv1D(256, 12, 3, ), # hidden
#     keras.layers.Conv1D(128, 6, 3, ), # hidden
#     keras.layers.Flatten(),
#     keras.layers.Dense(1024, activation="relu"), # output
#     keras.layers.Dropout(0.5),
#     keras.layers.Dense(vertices)
# ])
model.compile(optimizer="adam", loss=keras.losses.MeanSquaredError(), metrics="mse")

In [20]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 256)               62464     
                                                                 
 dense_5 (Dense)             (None, 256)               65792     
                                                                 
 dense_6 (Dense)             (None, 256)               65792     
                                                                 
 dense_7 (Dense)             (None, 7568)              1944976   
                                                                 
Total params: 2,139,024
Trainable params: 2,139,024
Non-trainable params: 0
_________________________________________________________________


### Set up Tensorboard

### Train Model

In [21]:
# train model
history = model.fit(X_train, y_train, epochs=200, batch_size=8)

Epoch 1/200
12/12 [==============================] - 1s 17ms/step - loss: 0.0332 - mse: 0.0332
Epoch 2/200
12/12 [==============================] - 0s 18ms/step - loss: 0.0140 - mse: 0.0140
Epoch 3/200
12/12 [==============================] - 0s 11ms/step - loss: 0.0018 - mse: 0.0018
Epoch 4/200
12/12 [==============================] - 0s 10ms/step - loss: 6.3265e-04 - mse: 6.3265e-04
Epoch 5/200
12/12 [==============================] - 0s 10ms/step - loss: 2.7779e-04 - mse: 2.7779e-04
Epoch 6/200
12/12 [==============================] - 0s 8ms/step - loss: 1.9552e-04 - mse: 1.9552e-04
Epoch 7/200
12/12 [==============================] - 0s 10ms/step - loss: 1.2075e-04 - mse: 1.2075e-04
Epoch 8/200
12/12 [==============================] - 0s 8ms/step - loss: 8.5338e-05 - mse: 8.5338e-05
Epoch 9/200
12/12 [==============================] - 0s 10ms/step - loss: 7.0980e-05 - mse: 7.0980e-05
Epoch 10/200
12/12 [==============================] - 0s 8ms/step - loss: 5.8837e-05 - mse: 5.8837e

### Test Accuracy

In [23]:
results = model.evaluate(X_test, y_test, batch_size=16)
print("test loss, test acc:", results)

2/2 [==============================] - 0s 0s/step - loss: 2.0090e-05 - mse: 2.0090e-05
test loss, test acc: [2.0090277757844888e-05, 2.0090277757844888e-05]


### Get Model

In [24]:
wab = model.get_weights()

In [25]:
# manual evaluation 2 hidden layers: input = [u, v, theta]
def manual_eval(input):
    w1 = wab[0]
    b1 = wab[1]
    w2 = wab[2]
    b2 = wab[3]
    w3 = wab[4]
    b3 = wab[5]

    z1 = keras.activations.tanh(np.matmul(np.array(input), w1) + b1)
    z2 = keras.activations.tanh(np.matmul(z1, w2) + b2)
    z3 = keras.activations.tanh(np.matmul(z2, w3) + b3)
    return z3.numpy()[0]

### Save to output file

In [26]:
# save weights and biases as buffer
output_buf = str(len(model.layers)) + "\n" # layer count

for i in range(0, len(wab)-1, 2): # iterate through layers
    weights = wab[i]
    output_buf += str(weights.shape[0]) + " " + str(weights.shape[1]) + "\n" # input size output size
    for neuron in weights: # iterate through
        for edges in neuron:
            output_buf += str(edges) + " "
    for biases in wab[i+1]:
        output_buf += str(biases) + " "
    output_buf = output_buf[:-1] + "\n" # ignore extraneous space and end line
    

In [27]:
with open("../resources/evaluators/model.txt", "w") as text_file:
    text_file.write(output_buf) # add more digits